In [1]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision
import numpy as np

In [2]:
tf.keras.backend.set_floatx('float32')
iris = load_iris()
X, y = iris["data"], iris["target"]

In [3]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2)

In [4]:
class RecallMetric(tf.keras.metrics.Metric):
    def __init__(self, class_value = 2, **kwargs):
        super().__init__(**kwargs) 
        self.class_value = class_value

    def confusion_matrix(self, y_true, y_pred):
        self.y_pred = tf.math.argmax(y_pred, axis =1)
        self.y_true = tf.cast((y_true == self.class_value), dtype=tf.int32)
        self.y_pred = tf.cast((self.y_pred == self.class_value), dtype=tf.int32)
        self.TP = tf.math.count_nonzero(self.y_pred * self.y_true)
        self.TN = tf.math.count_nonzero((self.y_pred - 1) * (self.y_true - 1))
        self.FP = tf.math.count_nonzero(self.y_pred * (self.y_true - 1))
        self.FN = tf.math.count_nonzero((self.y_pred - 1) * self.y_true)
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        self.confusion_matrix(y_true, y_pred)

    def result(self):
        return self.TP/(self.TP + self.FN)

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

In [5]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
tf.config.experimental_run_functions_eagerly(True)

net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(10, activation='relu', kernel_initializer='he_normal'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
net.compile(loss='sparse_categorical_crossentropy', optimizer="nadam", metrics=['acc', RecallMetric(0)])
net.fit(xtrain, ytrain, epochs = 100)

Train on 120 samples
Epoch 1/100
120/120 [==============================] - 0s 2ms/sample - loss: 5.7451 - acc: 0.3667 - recall_metric: 1.0000
Epoch 2/100
120/120 [==============================] - 0s 892us/sample - loss: 5.5451 - acc: 0.3667 - recall_metric: 1.0000
Epoch 3/100
120/120 [==============================] - 0s 808us/sample - loss: 5.3377 - acc: 0.3667 - recall_metric: 1.0000
Epoch 4/100
120/120 [==============================] - 0s 742us/sample - loss: 5.1165 - acc: 0.3667 - recall_metric: 1.0000
Epoch 5/100
120/120 [==============================] - 0s 750us/sample - loss: 4.8850 - acc: 0.3667 - recall_metric: 1.0000
Epoch 6/100
120/120 [==============================] - 0s 900us/sample - loss: 4.6490 - acc: 0.3667 - recall_metric: 1.0000
Epoch 7/100
120/120 [==============================] - 0s 742us/sample - loss: 4.4063 - acc: 0.3667 - recall_metric: 1.0000
Epoch 8/100
120/120 [==============================] - 0s 958us/sample - loss: 4.1654 - acc: 0.3667 - recall_metr

&lt;tensorflow.python.keras.callbacks.History at 0x10d70408&gt;